In [13]:
from typing import List
import numpy as np

In [14]:
from fastembed import TextEmbedding

In [15]:
documents: List[str] = [
    "FastEmbed is lighter than Transformers & Sentence-Transformers.",
    "FastEmbed is supported by and maintained by Qdrant.",
]

In [16]:
embedding_model = TextEmbedding()
print("The model BAAI/bge-small-en-v1.5 is ready to use.")

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

C:\Users\HP\.conda\envs\nlp_env\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\HP\AppData\Local\Temp\fastembed_cache\models--qdrant--bge-small-en-v1.5-onnx-q. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/706 [00:00<?, ?B/s]

model_optimized.onnx:   0%|          | 0.00/66.5M [00:00<?, ?B/s]

2026-01-17 23:36:24.872 | ERROR    | fastembed.common.model_management:download_model:444 - Could not download model from HuggingFace: [WinError 1314] A required privilege is not held by the client: '..\\..\\blobs\\75305659f7795d4549f0e23688b52fa20a32f925' -> 'C:\\Users\\HP\\AppData\\Local\\Temp\\fastembed_cache\\models--qdrant--bge-small-en-v1.5-onnx-q\\snapshots\\52398278842ec682c6f32300af41344b1c0b0bb2\\tokenizer_config.json' Falling back to other sources.
2026-01-17 23:36:24.872 | ERROR    | fastembed.common.model_management:download_model:467 - Could not download model from either source, sleeping for 3.0 seconds, 2 retries left.


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

The model BAAI/bge-small-en-v1.5 is ready to use.


In [17]:
embeddings_generator = embedding_model.embed(documents)
embeddings_list = list(embeddings_generator)
len(embeddings_list[0])  

384

In [18]:
print("Embeddings:\n", embeddings_list)

Embeddings:
 [array([-9.47419927e-02,  1.01043023e-02, -3.08790691e-02,  2.37654261e-02,
        2.37953989e-03,  6.31102594e-04, -8.25395659e-02,  8.80879408e-04,
        3.71934250e-02,  1.43738203e-02, -2.38853190e-02,  8.76716455e-04,
        3.01597118e-02,  4.09234241e-02,  4.36942801e-02,  8.63894541e-03,
        5.05681373e-02, -1.26620168e-02, -3.13852839e-02,  2.06082482e-02,
        8.12607035e-02, -5.42182103e-02,  1.02508385e-02, -4.58789952e-02,
       -7.66648119e-03, -3.52218524e-02, -2.35789269e-02, -2.68959627e-02,
       -2.74154972e-02, -1.57299414e-01,  1.25554455e-02, -3.55682112e-02,
        6.45289943e-02, -1.35379005e-03, -3.40229236e-02,  9.82455257e-03,
       -3.25042829e-02,  1.50398910e-02, -6.96444213e-02, -1.67716779e-02,
       -2.34723557e-02,  2.00087838e-02, -1.55860698e-02,  1.41706679e-03,
       -1.32481596e-02, -2.17139274e-02,  6.85787052e-02, -2.02618912e-02,
       -9.10519529e-03, -1.48933558e-02,  6.37297109e-02,  1.35945166e-02,
        7.0

In [24]:
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance

client = QdrantClient(path="./qdrant_data")

In [26]:
COLLECTION = "source-of-truth"

if not client.collection_exists(COLLECTION):
    client.create_collection(
        collection_name=COLLECTION,
        vectors_config=VectorParams(
            size=384,
            distance=Distance.COSINE
        )
    )

In [31]:
#uploading to qdrant with payload

from qdrant_client.models import PointStruct

points = [
    PointStruct(
        id=i,
        vector=embeddings[i],
        payload={
            "text": documents[i],
            "source": "demo"
        }
    )
    for i in range(len(documents))
]

client.upsert(
    collection_name="source-of-truth",
    points=points
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [35]:
query = "lighter than transformer"
query_vector = list(embedding_model.embed([query]))[0]

results = client.query_points(
    collection_name="source-of-truth",
    query=query_vector,
    limit=3,
    with_payload=True
).points

for r in results:
    print(r.payload, r.score)

{'text': 'FastEmbed is lighter than Transformers & Sentence-Transformers.', 'source': 'demo'} 0.5201467959736713
{'text': 'FastEmbed is supported by and maintained by Qdrant.', 'source': 'demo'} 0.5007766544276349
